In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ****Projeto SQL Curso EBAC****

O Dataset escolhido está disponível no [link](https://www.kaggle.com/datasets/sufyant/brazilian-real-bank-dataset) . São informações de gasto de cartão de crédito de clientes brasileiros entre janeiro de 2019 e abril de 2020.

Eu retirei algumas colunas, que penso não ser tão relevantes para o presente projeto, para facilitar a nossa análise.

****Descrição dos dados****:

* id = id cliente
* safra_abertura = número da agência
* cidade = cidade da agência
* estado = estado da agência
* idade = idade do cliente
* sexo = gênero do cliente
* limite_total = limite do cartão de crédito
* limite_disp = limite ainda disponível do cartão
* valor = da compra
* grupo_estabelecimento = tipo de estabelecimento da compra
* SERVIÇO = qualquer serviço além de restaurante
* FAMARCIAS = farmácia
* Online (modifiquei o nome original para melhor compreensão) = compras online
* ARTIGOS ELETRO = eletrônicos
* VAREJO = loja de varejo
* LOJA DE DEPART = loja de departamento
* VESTUARIO = roupas
* HOSP E CLINICA = gastos médicos
* SEM RAMO = não especificado
* SUPERMERCADOS = mercado
* MAT CONSTRUCAO = material de construção
* POSTO DE GAS = posto de gasolina
* RESTAURANTE = restaurante
* CIA AEREAS = passagem aérea
* MOVEIS E DECOR = móveis e decoração
* JOALHERIA = jóias
* TRANS FINANC = transferência de valores
* AGENCIA DE TUR = agência de turismo
* HOTEIS = hoteis
* AUTO PEÇAS = peças de carros/mecânico 
* cidade_estabelecimento = cidade que a compra foi feita
* pais_estabelecimento = país que a compra foi feita


Os dados foram trabalhados no ****AWS Athena**** e ****S3 Bucket****.

# ****Exploração e Análise de dados****

Para enterdemos melhor o Dataset que temos em mão é necessário explorarmos os dados disponíveis. Utilizarei queries no AWS Athena com objetivo de amostrar o material que estamos trabalhando.

****Quantas linhas de informação temos nesse Dataset?**** 

* *Query 1*: select count(*) from cartao_credito

![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query1.jpg?raw=true)

O nosso Dataset tem 4955 linhas de dados, podemos traduzir isso para 4955 transações. Como projeto de estudo é um Dataset relativamente robusto, onde nós podemos tirar alguns insights interessantes para tentarmos entender padrões de consumo.

****Vamos selecionar apenas as 10 primerias linhas para ter uma visualização da nossa base de dados.**** 

*Query 2*: SELECT * FROM "default"."cartao_credito" limit 10

![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query2%20novo.jpg?raw=true)

****Quais os tipos de dados que temos?****

*Query 3*: describe cartao_credito

![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query3.jpg?raw=true)

Talvez seja interessante analisar todas as colunas não numéricas.

****Primeiro vamos analisar quais as cidades dos correntistas.**** 

*Query 4*: select distinct cidade from cartao_credito


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query4.jpg?raw=true)


De acordo com os resultados podemos ver que tirando a cidade do Rio de Janeiro, todas as agências são do estado de São Paulo, tal informação já elimina uma query que seria ****select distinct cidade from cartao_credito****, contudo já sabemos que os estados são São Paulo e Rio de Janeiro. 

****Vamos verificar quantas entradas tem cada cidade****

*Query 5*: select cidade, count (cidade) as distinct_cidade 
from cartao_credito 
group by cidade

![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query5.jpg?raw=true)



A cidade de São Paulo e Rio de Janeiro são as cidades mais representadas em nossos dados

****Agora vamos verificar os tipos de estabelecimento****

*Query 6*: select distinct grupo_estabelecimento from cartao_credito


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query6.jpg?raw=true)

Pela primeira vez no dataset podemos verificar a existência de dados nulos. ****Vamos verificar quantos valores temos de cada categoria, incluindo quantos nulos nós temos na base de dados****

*Query 7*: select grupo_estabelecimento, count (grupo_estabelecimento) as distinct_grupo_estabelecimento 
from cartao_credito 
group by grupo_estabelecimento

![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query7.jpg?raw=true)

Serviço e restaurante são as categorias de estabelecimento que mais aparecem em nosso Dataset, com apenas 4 valores nulos. Em relação ao totalm os 4 valores nulos não atrapalham nossa análise. 

****A próxima coluna a ser analisada é a cidade onde o cartão foi gasto****

*Query 8*: select distinct cidade_estabelecimento from cartao_credito

![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query8.jpg?raw=true)

Infelizmente a lista de cidades é muito extensa e não é possível colocar a imagem da lista, contudo é possível verificar que temos campos com nomes aleatórios nas cidades. ****Para provar que o número de ciades únicas é extenso, utilizei outra query****.

*Query 9*: select count(distinct cidade_estabelecimento) from cartao_credito


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query9.jpg?raw=true)

****Podemos checar quais foram as cidades que mais houve gastos do cartão****

*Query 10*: select cidade_estabelecimento, count (cidade_estabelecimento) as distinct_cidade_estabelecimento from cartao_credito group by cidade_estabelecimento

![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query19.jpg?raw=true)

Como era de se esperar, São Paulo e Rio de Janeiro são as cidades da maioria das transações.

Para finalizar os dados não numéricos, ****vamos analisar em quais países foram feitas compras****

*Query 11*: select distinct pais_estabelecimento from cartao_credito


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query10.jpg?raw=true)

O Brasil não é o único país de origem do estabelecimento, três entradas chamam atenção: OH, que leva a crer que seja o estado de Ohio nos Estados Unidos da América, SP que está como país e também o fato de termos valores nulos nessa coluna. ****Vejamos quantas entradas temos de cada país, incluindo o nulo****


*Query 12*: select pais_estabelecimento, count (pais_estabelecimento) as distinct_pais_estabelecimento
from cartao_credito 
group by pais_estabelecimento


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query11.jpg?raw=true)


O Brasil é o país de quase todas as transações, seguido de surpreendetemente valores nulos e só em terceiro lugar Estados Unidos da América.


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/IMG-20241107-WA0017.jpg?raw=true)




****Vamos analisar o número de Homens e Mulheres entre os clientes****

*Query 13*: select count(*), sexo from cartao_credito group by sexo


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query12.jpg?raw=true)

A diferença entre homens e mulheres é mínima, para ilustrar melhor vamos fazer outro gráfico

![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/sexo.jpg?raw=true)

Como podemos ver as mulheres são 53% da nossa base de dados e homens 47%, não temos valores nulos.


****O próximo passo é analisar a idade atrelada ao sexo****

*Query 14*: select avg(idade) as media_idade, min(idade) as min_idade, max(idade) as max_idade, sexo from cartao_credito group by sexo


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query13.jpg?raw=true)


As idades são bem parecidas entre os sexos, talvez a maior discrepância é a idade mínima das mulheres com 20 anos e 27 para homens. 


****Maior e menor valor****

*Query 15*: select min(valor) as menor_compra, max(valor) as maior_compra from cartao_credito


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query14.jpg?raw=true)

Esse valor mínimo deve ter sido algum erro na hora do pagamento ou apenas uma checagem online do cartão de crédito, ****vamos verificar mais informações dessa transação****

*Query 16*: select * from cartao_credito
 where valor = (select min(valor) from cartao_credito)


 ![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query15.jpg?raw=true)

 Essa transação está relacionada a algum estabelecimento de serviço na cidade de São Paulo, se fosse online explicaria o valor. Não podemos inferir muita coisa com essas informações. 

 ****Vamos verificar mais informações da maior transação****

 *Query 17*: select * from cartao_credito
 where valor = (select max(valor) from cartao_credito)

 ![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query16.jpg?raw=true)

 Esse valor de 14618.36 na categoria de passagem aérea faz muito mais sentido, considerando que pode ser uma passagem de classe executiva para o exterior. Podemos ver também que o cliente, naturalmente, tem um limite alto no cartão de crédito de 40 mil reais e ainda, no momento da compra, tinha o limite disponível de 21748 reais, levando em conta a idade podemos inferir que é um empresário. 

****Pode ser interessante verificar as maiores compras por categoria****

*Query 18*: select grupo_estabelecimento, max(valor)
from cartao_credito
group by grupo_estabelecimento


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query20.jpg?raw=true)

Se desconsiderarmos o "inexistente", nós temos companhia aérea, restaurante e serviços como maiores gastos com cartão de crédito.

****Vamos verificar informações das 10 maiores compras****

*Query 19*: select max(valor) as maior_compra,id, limite_total, limite_disp, sexo, idade, grupo_estabelecimento from cartao_credito 
group by id, limite_total, limite_disp, sexo, idade, grupo_estabelecimento order by max(valor) desc limit 10


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query17.jpg?raw=true)


Como podemos ver, 6 das 10 maiores compras por cartão de crédito é do mesmo cliente, homem de 43 anos. Os maiores gastos são com passagem aérea. Podemos inferir que realmente é um empresário que viaja com certa frequência para gastar esses valores com passagem aérea. Dessas 10 maiores transações apenas 2 são de mulheres com mecânico e passagem aérea. 

Outra informação que podemos tirar da query é que o limite total tem influência no valor da compra, contudo temos alguns casos de limites menores com gastos maiores do que outros clientes com limite de crédito maior, como é o caso das mulheres. Será que tem alguma explicação para mulheres terem gastos altos com limites relativamente menores?

Já vimos que a diferença entre o maior valor gasto entre homens e mulheres é bem discrepante, mas será que na média também temos essa diferença? ****Vamos ver os valores gastos por sexo****

*Query 20*: select max(valor) as maior_valor, avg(valor) as media_valor, min(valor) as min_valor, sexo from cartao_credito group by sexo


![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query18.jpg?raw=true)


A média de gastos também tende para os homens, provavelmente por causa das maiores transações serem de homens. 

****Uma última análise que podemos fazer é o limite máximo de crédito de acordo com o sexo****

*Query 21*: select sexo, max(limite_total) as limite_total 
from cartao_credito
group by sexo

![](https://github.com/guiaraujo017/ImagensEbacSQL/blob/main/query21.jpg?raw=true)


A diferença de limite do cartão de crédito é bem discrepante entre os sexos, sendo 40 mil para homens e 10.500 para mulheres. 

# ****Conclusão****

Aguns pontos que sabemos analisando os dados:

1. Esse banco estudado possui agência apenas no estado de São Paulo e Rio de Janeiro;
2. A maior parte das agências ficam na cidade de São Paulo e Rio de Janeiro;
3. As cidades com mais gastos são, mais uma vez, São Paulo e Rio de Janeiro; 
4. Os estabelecimentos onde o cartão é mais utilizado são: Serviço, Restaurante, Varejo e Supermercado;
5. Os países com mais gastos, excluindo os valores nulos, são Brasil, Estados Unidos e Chile;
6. Temos mais mulheres do que homens como clientes
7. As idades entre sexo são bem parecidas entre 20 e 53 anos, a maior diferença está na idade mínima que é de 20 para mulheres e 27 para homens;
8. Apesar de idades e número de clientes parecidos, os valores das transações possuem uma diferença considerável com as maiores transações sendo do sexo masculino e apenas 2 transações de mulheres no top 10;
9. O maior valor gasto em cada categoria


Essas informações podem ser valiosas para a empresa de cartão de crédito de diferente formas. Por exemplo, que tipo de benefício a empresa pode oferecer para cada tipo de cliente? para cada tipo de estabelecimento? É possível oferecer algum benefício para compras no exterior?

Também é possível fazer um estudo para entender por qual motivo as mulheres gastam menos ou até mesmo por qual motivo possuem menos crédito. Uma campanha pode ser feita para atrair mulheres com maior poder de compra.

Com essas informações é possível tomar decisões para consolidar áreas que estão bem estabelecidas, como o cliente homem e os gastos com serviços, contudo com esse estudo também é possível trabalhar em áreas que necessitam de mais atenção como clientes mulheres com maior poder de compras e uso do cartão no exterior por exemplo. 